In [1]:
from rdflib import Graph
from pydantic import BaseModel, validator, Field
from typing import Optional
from pyfuseki import FusekiUpdate
import json
from datetime import date, datetime

In [2]:
class GenerationProcess(BaseModel):
    label: str = Field(default="BiblioKeia v.1")
    generationDate: Optional[str]

    @validator('generationDate', pre=True, always=True)
    def set_date_now(cls, v):
        now = datetime.now().strftime('%d-%m-%YT%H:%M:%S')
        
        return now

In [18]:
class AdminMetadata(BaseModel):
    encodingLevel: str = Field(default="menclvl:f")
    assigner: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    creationDate: Optional[str]
    descriptionConventions: str = Field(default="http://id.loc.gov/vocabulary/descriptionConventions/isbd")
    descriptionLanguage: str = Field(default="http://id.loc.gov/vocabulary/languages/por")
    generationProcess: Optional[GenerationProcess]
    status: str = Field(default="mstatus:new")

    _status = [
        "mstatus:cancinv", 
        "mstatus:ceased", 
        "mstatus:c", 
        "mstatus:current", 
        "mstatus:d", 
        "mstatus:former", 
        "mstatus:incorrect",
        "mstatus:new"
        ]
    

    _level = [
        "menclvl:1", 
        "menclvl:3", 
        "menclvl:4", 
        "menclvl:5", 
        "menclvl:7", 
        "menclvl:8", 
        "menclvl:f"]

    @validator('status')
    def status_accepted(cls, v):
        if v not in cls._status:
            raise ValueError(f"the status code must be one of the following : {', '.join(cls._status)}")
        return v
    
    @validator('encodingLevel')
    def level_accepted(cls, v):
        if v not in cls._level:
            raise ValueError(f"the level code must be one of the following : {', '.join(cls._level)}")
        return v

    @validator('creationDate', pre=True, always=True)
    def set_date_today(cls, v):
        today = date.today().strftime("%d-%m-%Y")
        
        return today

In [15]:
class Work(BaseModel):
    type: list[str]
    adminMetadata: AdminMetadata
    # content: str
    # title: Title
    # primaryContribution: Optional[Contribution]
    # subjects: list[Subject]
    # language: str
    # languageCode: str 
    # cdd: str
    # cutter: str
    # serie: Optional[str]
    # serieURI: Optional[str]

In [19]:
obj = {
    "type": [
        "bf:Monograph", "bf:Text", "bf:Work"
    ],
    "adminMetadata": {
        "encodingLevel": "menclvl:1",
        "generationProcess": {
            "label": "BiblioKeia"
        }
    }
}
request = Work(**obj)
request

Work(type=['bf:Monograph', 'bf:Text', 'bf:Work'], adminMetadata=AdminMetadata(encodingLevel='menclvl:1', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='14-04-2023', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess=GenerationProcess(label='BiblioKeia', generationDate='14-04-2023T16:04:47'), status='bf:new'))

In [71]:
request.adminMetadata

AdminMetadata(encodingLevel='menclvl:1', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='2023-04-14')

In [17]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    """

In [75]:
request.adminMetadata.creationDate

'14-04-2023'

In [20]:
id = "bk-2"
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH bkw:{id}
        {{ 
           bkw:{id} a { ", ".join(request.type) }  ;
           bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel {request.adminMetadata.encodingLevel} ;
            bf:assigner <{request.adminMetadata.assigner}> ;    
            bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;    
            bf:descriptionConventions <{request.adminMetadata.descriptionConventions}> ;
            bf:descriptionLanguage <{request.adminMetadata.descriptionLanguage}> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess.label}" ;
                    bf:generationDate "{request.adminMetadata.generationProcess.generationDate}"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <{request.adminMetadata.assigner}> ;
                    rdf:value "{id}" ] ;
            bf:status {request.adminMetadata.status}
            ] ;
    
        }} }}"""

print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        
    INSERT DATA {
        GRAPH bkw:bk-2
        { 
           bkw:bk-2 a bf:Monograph, bf:Text, bf:Work  ;
           bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel menclvl:1 ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;    
            bf:creationDate "14-04-2023"^^xsd:date ;    
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/isbd> ;
            bf:descriptionLanguage <htt

 ns1:assigner <http://id.loc.gov/vocabulary/organizations/dlc> ;
            ns1:changeDate "2023-02-06T08:03:19"^^xsd:dateTime ;
            ns1:creationDate "2019-11-26"^^xsd:date ;
            ns1:descriptionAuthentication <http://id.loc.gov/vocabulary/marcauthen/pcc> ;
            ns1:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/isbd>,
                <http://id.loc.gov/vocabulary/descriptionConventions/rda> ;
            ns1:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            ns1:descriptionModifier <http://id.loc.gov/vocabulary/organizations/dlc> ;
            ns1:generationProcess [ a ns1:GenerationProcess ;
                    rdfs:label "DLC marc2bibframe2 v2.2.1" ;
                    ns1:generationDate "2023-02-07T01:58:10.983465-05:00"^^xsd:dateTime ] ;
            ns1:identifiedBy [ a ns1:Local ;
                    ns1:assigner <http://id.loc.gov/vocabulary/organizations/dlc> ;
                    rdf:value "21313495" ] ;
            ns1:status <http://id.loc.gov/vocabulary/mstatus/c> ;

In [13]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

response = fuseki_update.run_sparql(graph)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exemple Work Loc

In [2]:
g = Graph()
g.parse('http://id.loc.gov/resources/works/21313495.nt', format='nt')

<Graph identifier=Nca383953bce9402fb514c1e486a66fae (<class 'rdflib.graph.Graph'>)>

In [3]:
g.serialize('work.ttl', format='ttl')

<Graph identifier=Nca383953bce9402fb514c1e486a66fae (<class 'rdflib.graph.Graph'>)>

POST WORK